In [1]:
# 핑퐁 로봇 제어 모듈 import.
from pingpongthread import PingPongThread
from time import sleep
from keyboard import is_pressed

In [2]:
# 1개 로봇을 연결하는 인스턴스 생성.
PingPong = PingPongThread(number=2, group_id=46)
# 로봇 제어 쓰레드 시작.
PingPong.start()

Found device: USB 직렬 장치(COM14)
Serial connected.
Write data: DD DD 00 DD 00 00 DA 00 0B 00 00


In [7]:
# 웹캠 열기.
PingPong.webcam_open(0)
#PingPong.webcam_open("http://192.168.66.1:9527/videostream
#                                   .cgi?loginuse=admin&loginpas=admin")

In [ ]:
# trainingData/7.no_mask 폴더 안에 '마스크 안 쓴 사진' 찍기.
PingPong.webcam_take_snapshots("trainingData/7.no_mask", 0.5)

In [ ]:
# trainingData/7.with_mask 폴더 안에 '마스크 쓴 사진' 찍기.
PingPong.webcam_take_snapshots("trainingData/7.with_mask", 0.5)

In [5]:
# '마스크 안 쓴 사진' 클래스 인스턴스, ''마스크 쓴 사진' 클래스 인스턴스 생성.
no_mask = PingPong.ImageClass("no_mask", "trainingData/7.no_mask")
with_mask = PingPong.ImageClass("with_mask", "trainingData/7.with_mask")

In [6]:
# 모델 트레이닝.(학습 시키기)
# 첫 번째 인수는 저장하는 이름, 두 번째 인수는 knn 알고리즘의 k 값,
# 세 번째 인수는 모델 모드. 네 번째 인수 이후는 클래스 인스턴스들.
model = PingPong.train_classes("model/7.mask_model.json", 5, 1, no_mask, with_mask)

Class no_mask : 42 images.
Class with_mask : 17 images.
2/2 [==============================] - 2s 670ms/step
Training done.
model/7.mask_model.json saved.


In [8]:
# 프레임을 평가하는 인스턴스 생성. 누적 프레임은 1초 동안 보관.
frames_predictor = PingPong.FramesPredictor(model=model, timer_sec=3)
# 웹캠을 이용하여 마스크 착용 여부를 예측하는 루프.
while not is_pressed('esc'):
    # 주피터 노트북 출력 비우기.
    PingPong.clear_output()
    # 현재 웹캠 프레임 가져오기.
    frame = PingPong.webcam_get_frame(window="Get_frame")
    # 현재 프레임을 평가하고, 평가 내용을 누적.
    frames_prediction = frames_predictor.image_predict_and_accum(frame)
    print(frames_prediction)
    
    sleep(0.1) # 0.1초 기다리기.
    #가장 확률이 높은 클래스 이름을 max_class에 저장.
    max_class= max(frames_prediction, key=frames_prediction.get)
    print(max_class) # 가장 높은 확률의 클래스 이름 출력.

KeyboardInterrupt: 

In [ ]:
# 웹캠 닫기 및 핑퐁 스레드 종료
PingPong.webcam_close()
PingPong.end()